RAG-BASED LLM

In [1]:
!pip install -U -q langchain langchain-google-genai langchain-chroma langchain-community gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 927.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.4/719.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/27

In [3]:
import pandas as pd
df = pd.read_csv("reviews.csv")
print(df.columns)

Index(['review_id', 'visit_id', 'review', 'physician_name', 'hospital_name',
       'patient_name'],
      dtype='object')


In [28]:
from IPython.terminal.embed import embed
from inspect import formatargvalues
import time
from google.colab import userdata
from langchain_community.document_loaders import CSVLoader
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/text-embedding-004",
    google_api_key = userdata.get('GEMINI')

)

loader = CSVLoader(file_path="reviews.csv", source_column="review")
docs = loader.load()

vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./patient_db"
)

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata # Import userdata
from langchain_google_genai import ChatGoogleGenerativeAI # Import ChatGoogleGenerativeAI

template = """
You are a Senior Hospital Operations Consultant.
Your goal is to analyze patient reviews to help management improve services.

STRICT RULES:
1. Use ONLY the provided context.
2. If the answer isn't there, say "Data not available in current reviews."
3. Cite the 'Source' (Review Content) if possible.

CONTEXT:
{context}

QUESTION:
{question}

ANALYSIS:
"""

prompt = ChatPromptTemplate.from_messages([("human", template)])
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=userdata.get('GEMINI'), # Provide the API key
    max_retries=6
)

def respond_to_user_question(question: str, history: list) -> str:
    try:
        return rag_chain.invoke(question)
    except Exception as e:
        if "429" in str(e):
            return "Server is busy (Rate Limit hit). Please wait a moment and try again!"
        return f"An error occurred: {str(e)}"


In [41]:
def format_docs(docs):
  return "\n\n".join([f"Review: {d.page_content}" for d in docs])

rag_chain = (
    {"context": vector_db.as_retriever(k=5) | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
# API check karne ke liye simple test
try:
    test_response = llm.invoke("Hi, are you working?")
    print("API is working fine:", test_response.content)
except Exception as e:
    print("API Error found:", e)

API is working fine: Yes, I am! As a large language model, I don't really "stop" or "start" working in the human sense. I'm always available and ready to process information and respond to your queries.

So, yes, you could say I'm "working" right now by interacting with you. How can I help you today?


Please run the above cell and share the output so I can help you pick a supported model name for `ChatGoogleGenerativeAI`.

In [43]:
import gradio as gr

def chat_function(message, history):
    # Logic to handle the conversation
    response = rag_chain.invoke(message)
    return response

# Customizing the UI for your CV
demo = gr.ChatInterface(
    fn=chat_function,
    title="🏥 Patient Experience Insights Engine",
    description="Analyze hospital reviews using RAG (Retrieval-Augmented Generation). Built with Gemini 1.5 & LangChain.",
    examples=["What are the main complaints about wait times?", "Was the discharge process smooth?"]
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://174b022858e5e2158b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
